In [8]:
# this program is inspired by "Quantlib Python Cookbook" by Balaraman and Ballabio - Probando

from QuantLib import *
today = Date(21, September, 2021)
Settings.instance().evaluationDate = today

quotes = [SimpleQuote(2.80763/100) ]

helpers = [DepositRateHelper(QuoteHandle(quotes[0]), Period(3,Months), 3, China(), ModifiedFollowing, False, Actual360())]

for rate, tenor in [(2.6569, 2), (2.5899, 3), (2.5682, 4), (2.5703, 5),
                    (2.5969, 6), (2.6235, 7), (2.6493, 8), (2.6828, 9),
                    (2.7084, 10), (2.7309, 11), (2.7601, 12), (2.8044, 15)]:

    quotes.append(SimpleQuote(rate/100))
    helpers.append(SwapRateHelper(QuoteHandle(quotes[-1]),
                   Period(tenor, Years), China(),
                   Annual, Unadjusted, Thirty360(Thirty360.BondBasis),USDLibor(Period(3,Months))))

rate_curve = PiecewiseLogCubicDiscount(2, UnitedStates(), helpers, Actual365Fixed())
curve_handle = RelinkableYieldTermStructureHandle(rate_curve)


fixed_schedule = Schedule(Date(21, October, 2021), Date(21, October, 2031),
Period(1, Years), China(), ModifiedFollowing, ModifiedFollowing,
DateGeneration.Forward, False)

floating_schedule = Schedule(Date(21, October, 2021), Date(21, October, 2031),
Period(6, Months), China(), ModifiedFollowing, ModifiedFollowing,
DateGeneration.Forward, False)

index = USDLibor(Period(3,Months),curve_handle)
swap = VanillaSwap(VanillaSwap.Payer, 10000000.0,fixed_schedule, 2.5/100, Thirty360(), floating_schedule, index, 0.0, Actual360())

swap.setPricingEngine(DiscountingSwapEngine(curve_handle))
P0 = swap.NPV()
#to RMB 
print("Swap NPV=", P0*0.15)


Swap NPV= 27100.33342906467
